In [26]:
# 딥러닝 구동하는 데 필요한 kearas 함수 호출
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# 필요한 라이브러리 불러옴
import numpy as np
import tensorflow as tf

# 실행할 때마다 같은 결과를 출력하기 위해 설정하는 부분
np.random.seed(3)
tf.random.set_seed(3)

# 준비된 수술 환자 데이터를 불러옴
Data_set = np.loadtxt("../../dataset/ThoraricSurgery.csv",delimiter=",") # 구분인자: , <- 없으면 파일 인식 못 함


# 데이터의 구분
| | 정보 1 | 정보 2 | 정보 3 | ... | 정보 17 | 생존 여부 |
|---|---|---|---|---|---|---|
| 환자1 | | | | | | 0 |
| 환자2 | | | | | | 0 |
| 환자3 | | | | | | 1 |
| ... | | | | | |  |
| 환자470 | | | | | | 1 |

데이터셋은 현재 위의 표와 같다. 이때 데이터를 속성(feature), 클래스, 샘플로 구분할 수 있다.
* 속성(attribute)/feature: 정보 1~정보 17에 해당하는 부분
* 샘플/instance: 환자1~환자470에 해당하는 부분
* 클래스(class): 생존 여부에 해당하는 부분

In [27]:
# 환자의 기록과 수술 결과를 X와 Y로 구분하여 저장
X=Data_set[:,:17] # 모든 행에서 0~17 열 가져 옴 -> 470 x 17
Y=Data_set[:,17] # 모든 행에서 17번 열 가져 옴 -> 470 x 1
### cf. 리스트 슬라이싱 (https://m.blog.naver.com/jjys9047/221566766122)

# 1. 딥러닝 구조 결정: 입력층, 은닉층, 출력층
``` python
model.add(Dense(30, input_dim=17, activation='relu'))
```
* add(): 새로운 층(layer) 추가
* Dense(): 이 층에 몇 개의 노드 만들 것인지
* input_dim: 입력 데이터에서 몇 개의 값을 가져올지
kears는 입력층을 따로 만드는 것이 아니라, 첫 번째 은닉층에 input_dim을 적어줘서 첫 번째 Dense가 은닉층 + 입력층 역할을 하게 한다.

위의 코드는 데이터에서 17개의 값을 입력으로 받아 은닉층의 30개 노드로 전달한다는 의미이고 그림으로 표현하면 아래와 같다.


![은닉층 Dense](https://user-images.githubusercontent.com/78032658/150663492-0c37db84-8510-4716-905d-a95858410860.png)


In [28]:
# 딥러닝 구조를 결정(모델을 설정하고 실행하는 부분)
model = Sequential()
### Sequential 모델: 각 레이어에 정확히 하나의 입력 텐서와 하나의 출력 텐서가 있는 일반 레이어 스택에 적합한 TensorFlow 모델
### cf. Sequential 모델 소개 (https://www.tensorflow.org/guide/keras/sequential_model?hl=ko)
model.add(Dense(30, input_dim=17, activation='relu')) # 층 +1 -> 은닉층, Dense
model.add(Dense(1,activation='sigmoid')) # 층 +1 -> 출력층(출력 값을 하나로 정해서 보여 줘야 하므로 노드 1개)


17개의 입력 값은 은닉층의 각 노드로 들어가서 임의의 가중치를 갖고 활성화 함수에 들어가게 된다. 그리고 활성화 함수를 거친 결괏값이 출력층으로 전달된다.   
Ex. 아래의 수식은 입력 값이 2개이고, 은닉층의 노드가 2개일 때 하나의 출력 값 y의 계산이다.


$$ n_{1} = \sigma(x_{1}w_{11}+x_{2}w_{21}+b_{1}) $$
$$ n_{2} = \sigma(x_{1}w_{12}+x_{2}w_{22}+b_{2}) $$
$$ y_{out} = \sigma(n_{1}w_{31}+n_{2}w_{32}+b_{3}) $$

# 2. 모델 컴파일 
모델을 컴파일 한다. 앞서 지정한 모델의 구현을 위해서 여러 가지 환경을 결정 및 설정하여 컴파일하는 부분이다.   
1. 오차 함수 -> 평균 제곱 오차 함수(mean_squared_error)
2. optimizer(최적화 방법) -> adam
3. metrics(): 모델이 컴파일될 때 모델 수행 결과를 나타내도록 설정하는 부분. 테스트 샘플을 학습 과정에서 제외시켜서 overfitting 방지   
(테스트 샘플은 정확도를 측정하기 위해 사용된다.)

![폐암 환자 생존율 예측 신경망 모델 도식화](https://user-images.githubusercontent.com/78032658/150664470-8d036bd7-0974-42d3-bb19-14f50090fe71.png)

In [29]:
# 딥러닝 실행
model.compile(loss='mean_squared_error',optimzer='adam',metrics=['accuracy'])
### 위에서 정해진 모델을 컴퓨터가 알아들을 수 있게끔 컴파일

# 3. 딥러닝 모델 실행
앞서 컴파일 단계에서 설정한 환경에서 데이터를 불러와 딥러닝을 실행시킨다.  
* epochs: 각 샘플이 해당 수 만큼 재사용될 때까지 반복해라.
* batch_size: 한 번에 처리하는 샘플의 개수.너무 크면 학습 속도가 느려지고, 너무 작으면 각 실행 값의 편차가 생겨서 전체 결괏값이 불안정해짐. -> 자신의 컴퓨터 메모리에 적당한 batch_size 찾아 설정해주어야 함

In [30]:
model.fit(X,Y,epochs=100,batch_size=10)
### 모델을 실제로 수행하는 부분

Train on 470 samples
Epoch 1/100
470/470 [==============================] - 0s 671us/sample - loss: 0.1472 - accuracy: 0.8447
Epoch 2/100
470/470 [==============================] - 0s 66us/sample - loss: 0.1478 - accuracy: 0.8340
Epoch 3/100
470/470 [==============================] - 0s 68us/sample - loss: 0.1401 - accuracy: 0.8426
Epoch 4/100
470/470 [==============================] - 0s 68us/sample - loss: 0.1384 - accuracy: 0.8489
Epoch 5/100
470/470 [==============================] - 0s 74us/sample - loss: 0.1408 - accuracy: 0.8340
Epoch 6/100
470/470 [==============================] - 0s 76us/sample - loss: 0.1363 - accuracy: 0.8468
Epoch 7/100
470/470 [==============================] - 0s 77us/sample - loss: 0.1417 - accuracy: 0.8383
Epoch 8/100
470/470 [==============================] - 0s 77us/sample - loss: 0.1471 - accuracy: 0.8234
Epoch 9/100
470/470 [==============================] - 0s 70us/sample - loss: 0.1343 - accuracy: 0.8489
Epoch 10/100
470/470 [====================

470/470 [==============================] - 0s 77us/sample - loss: 0.1320 - accuracy: 0.8426
Epoch 77/100
470/470 [==============================] - 0s 75us/sample - loss: 0.1294 - accuracy: 0.8404
Epoch 78/100
470/470 [==============================] - 0s 73us/sample - loss: 0.1284 - accuracy: 0.8468
Epoch 79/100
470/470 [==============================] - 0s 74us/sample - loss: 0.1336 - accuracy: 0.8404
Epoch 80/100
470/470 [==============================] - 0s 77us/sample - loss: 0.1314 - accuracy: 0.8468
Epoch 81/100
470/470 [==============================] - 0s 73us/sample - loss: 0.1286 - accuracy: 0.8489
Epoch 82/100
470/470 [==============================] - ETA: 0s - loss: 0.1903 - accuracy: 0.70 - 0s 78us/sample - loss: 0.1308 - accuracy: 0.8489
Epoch 83/100
470/470 [==============================] - 0s 74us/sample - loss: 0.1281 - accuracy: 0.8426
Epoch 84/100
470/470 [==============================] - 0s 77us/sample - loss: 0.1277 - accuracy: 0.8404
Epoch 85/100
470/470 [====

# +) Cost Function(오차 함수)
오차 함수에 따라서 모델의 성능이 달라지기 때문에 적절한 오차 함수를 사용해줘야 한다.   
오차 함수의 종류는 평균 제곱 오차 계열과 교차 엔트로피 계열로 나누어진다.  
* 평균 제곱 오차 계열: 수렴하기까지 속도가 많이 걸린다.
* 교차 엔트로피 계열: 출력 값에 로그를 취하는 방법으로 오차가 커지면 수렴 속도가 빨리지고, 오차가 작아지면 수렴 속도가 감소하도록 한다. 주로 분류 문제에 많이 사용된다.

| 평균 제곱 계열 | mean_squared_error | 평균 제곱 오차 |
| --- | --- | --- | 
| | mean_absolute_error |  평균 절대 오차 | 
| | mean_absolute_percentage_error | 평균 절대 백분율 오차 |
| | mean_squared_logarithmic_error | 평균 제곱 로그 오차 |
| 교차 엔트로피 계열 | categorical_crossentropy | 범주형 교차 엔트로피(일반적인 분류) |
|  | binary_crossentropy | 이항 교차 엔트로피(두 개의 클래스 중에서 예측할 때) |
